# Analyzing the Solutions Obtained from Algorithm

In [1]:
import os
%load_ext autoreload
%autoreload 2

In [34]:
import Snowplow_Routing_Middleton.dual_graphs as dual_graphs
import Snowplow_Routing_Middleton.construction as construction
import Snowplow_Routing_Middleton.plotting as plotting
import Snowplow_Routing_Middleton.local_search as local_search
import Snowplow_Routing_Middleton.initialize as initialize
import Snowplow_Routing_Middleton.params as params
import Snowplow_Routing_Middleton.sectioning as sectioning
import Snowplow_Routing_Middleton.costs as costs


from Snowplow_Routing_Middleton.shortest_paths import ShortestPaths
from Snowplow_Routing_Middleton.crossover import apply_crossover
from Snowplow_Routing_Middleton.genetic import run_genetic
from Snowplow_Routing_Middleton.solution import Solution
from Snowplow_Routing_Middleton.costs import routes_cost, route_travel_time, calculate_time_deadheading
from Snowplow_Routing_Middleton.routes_representations import create_full_routes_with_returns
from Snowplow_Routing_Middleton.turns import turn_direction_count
from Snowplow_Routing_Middleton.main import create_section_graphs_instances

import pandas as pd
import osmnx as ox
import folium
import folium.plugins
import networkx as nx
import pickle
import matplotlib.pyplot as plt


In [3]:
curr_path = os.path.abspath("")
parent_path = os.path.split(curr_path)[0]

In [4]:
blue_polygon_path = os.path.join(parent_path, "graph_data\\individual_sections\\route1_blue.gpkg")
orange_polygon_path = os.path.join(parent_path, "graph_data\\individual_sections\\route2_orange.gpkg")
red_polygon_path = os.path.join(parent_path, "graph_data\\individual_sections\\route4_red.gpkg")
green_polygon_path = os.path.join(parent_path, "graph_data\\individual_sections\\route3_green.gpkg")

blue_shortestpaths_folder = os.path.join(parent_path, "graph_data\\shortest_paths\\route1_blue")
orange_shortestpaths_folder = os.path.join(parent_path, "graph_data\\shortest_paths\\route2_orange")
green_shortestpaths_folder = os.path.join(parent_path, "graph_data\\shortest_paths\\route3_green")
red_shortestpaths_folder = os.path.join(parent_path, "graph_data\\shortest_paths\\route4_red")

In [9]:
orange_solution_filepath = os.path.join(parent_path, "solutions\\solution_orange.pickle")
green_solution_filepath = os.path.join(parent_path, "solutions\\solution_green.pickle")
red_solution_filepath = os.path.join(parent_path, "solutions\\solution_red.pickle")
blue_solution_filepath = os.path.join(parent_path, "solutions\\solution_blue.pickle")

## Load graphs and shortest paths

In [25]:
G_BLUE, G_BLUE_DUAL, BLUE_SP, BLUE_DEPOT = create_section_graphs_instances(blue_polygon_path, blue_shortestpaths_folder, required_parts=False)
G_ORANGE, G_ORANGE_DUAL, ORANGE_SP, ORANGE_DEPOT = create_section_graphs_instances(orange_polygon_path, orange_shortestpaths_folder)
G_RED, G_RED_DUAL, RED_SP, RED_DEPOT = create_section_graphs_instances(red_polygon_path, red_shortestpaths_folder)
G_GREEN, G_GREEN_DUAL, GREEN_SP, GREEN_DEPOT = create_section_graphs_instances(green_polygon_path, green_shortestpaths_folder, plow_culdesacs=False)

Depot found, sections are done
Adding depot source 136
Graphs created. Depot is at 136
Depot found, sections are done
Adding depot source 310
Graphs created. Depot is at 310
Depot found, sections are done
Adding depot source 277
Graphs created. Depot is at 277
Depot found, sections are done
Adding depot source 274
Graphs created. Depot is at 274


## Load solutions

In [13]:
with open(blue_solution_filepath, 'rb') as f:
    blue_solution: Solution = pickle.load(f)
with open(orange_solution_filepath, 'rb') as f:
    orange_solution: Solution = pickle.load(f)
with open(green_solution_filepath, 'rb') as f:
    green_solution: Solution = pickle.load(f)
with open(red_solution_filepath, 'rb') as f:
    red_solution: Solution = pickle.load(f)

Travel times

In [43]:
cost = routes_cost(G_BLUE, BLUE_SP, blue_solution.routes, BLUE_DEPOT)
blue_full_routes = create_full_routes_with_returns(G_BLUE, BLUE_SP, blue_solution.routes, BLUE_DEPOT)
travel_time = route_travel_time(G_BLUE, blue_full_routes, BLUE_DEPOT)
deadheading_time = calculate_time_deadheading(blue_solution.routes, BLUE_SP, BLUE_DEPOT)

print("cost", cost)
print("travel_time", travel_time/3600, "hours")
print("deadheading_time", deadheading_time/3600, "hours")

cost 25535.08789768458
travel_time 6.675279794974631 hours
deadheading_time 1.3879427151119281 hours


In [41]:
cost = routes_cost(G_ORANGE, ORANGE_SP, orange_solution.routes, ORANGE_DEPOT)
orange_full_routes = create_full_routes_with_returns(G_ORANGE, ORANGE_SP, orange_solution.routes, ORANGE_DEPOT)
travel_time = route_travel_time(G_ORANGE, orange_full_routes, ORANGE_DEPOT)
deadheading_time = calculate_time_deadheading(orange_solution.routes, ORANGE_SP, ORANGE_DEPOT)

print("cost", cost)
print("travel_time", travel_time/3600, "hours")
print("deadheading_time", deadheading_time/3600, "hours")

Returning to Depot
cost 35982.679220472404
travel_time 8.117226923177734 hours
deadheading_time 1.7075995843365666 hours


In [35]:
cost = routes_cost(G_GREEN, GREEN_SP, green_solution.routes, GREEN_DEPOT)
green_full_routes = create_full_routes_with_returns(G_GREEN, GREEN_SP, green_solution.routes, GREEN_DEPOT)
travel_time = route_travel_time(G_GREEN, green_full_routes, GREEN_DEPOT)
deadheading_time = calculate_time_deadheading(green_solution.routes, GREEN_SP, GREEN_DEPOT)

print("cost", cost)
print("travel_time", travel_time/3600, "hours")
print("deadheading time", deadheading_time/3600, "hours")


cost 26119.294383788147
travel_time 6.329693129058135 hours
deadheading time 1.5779938850296091 hours


In [36]:
cost = routes_cost(G_RED, RED_SP, red_solution.routes, RED_DEPOT)
red_full_routes = create_full_routes_with_returns(G_RED, RED_SP, red_solution.routes, RED_DEPOT)
travel_time = route_travel_time(G_RED, red_full_routes, RED_DEPOT)
deadheading_time = calculate_time_deadheading(red_solution.routes, RED_SP, RED_DEPOT)

print("cost", cost)
print("travel_time", travel_time/3600, "hours")
print("deadheading time", deadheading_time/3600, "hours")

cost 29901.430962908584
travel_time 6.850872464466601 hours
deadheading time 1.2952310636564284 hours
